# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Загрузка данных

In [ ]:
!pip install catboost -q

In [ ]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector, make_column_transformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

from imblearn.combine import SMOTETomek
from imblearn.under_sampling import RandomUnderSampler, NearMiss, TomekLinks, EditedNearestNeighbours
from imblearn.base import FunctionSampler
from imblearn.pipeline import make_pipeline as make_imblearn_pipeline

from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# df = pd.read_csv('/content/drive/MyDrive/csv/toxic_comments.csv', index_col = 0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/datasets/toxic_comments.csv', index_col = 0)

In [ ]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


В данных, 2 колонки: 1-ая это текст представленный комментариями в типе данных object, 2-ая колонка это целевой признак бинарной классификации представленный в 1 или 0 как тип данных int

In [ ]:
df['toxic'].value_counts()/len(df)

,count
toxic,
0,0.898388
1,0.101612


Есть дисбаланс классов на 9 обычных комментариев встречается 1 'токсичный'

In [ ]:
df.isna().sum()

,0
text,0
toxic,0


Пропусков в данных не наблюдаем

In [ ]:
df.duplicated().sum()

0

In [ ]:
df['text'].duplicated().sum()

0

Явных дубликатов нет

In [ ]:
df['text'][:5]

,text
0,Explanation\nWhy the edits made under my usern...
1,D'aww! He matches this background colour I'm s...
2,"Hey man, I'm really not trying to edit war. It..."
3,"""\nMore\nI can't make any real suggestions on ..."
4,"You, sir, are my hero. Any chance you remember..."


In [ ]:
df['text'][0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

Лемматизируем текст:

In [ ]:
lemmatizer = WordNetLemmatizer()
text = df['text'][0]
lemm_text = lemmatizer.lemmatize(text)
print(lemm_text)

Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27


Обработаем его (избавимся от запятых, точек, приведем к единому регистру и тд):

In [ ]:
def clear_text(text):
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-zA-Z ]+", "", text).strip()
    text = text.lower()
    return text

df['text'] = df['text'].apply(clear_text)

In [ ]:
df['text'][0]

'explanation why the edits made under my username hardcore metallica fan were reverted they werent vandalisms just closure on some gas after i voted at new york dolls fac and please dont remove the template from the talk page since im retired now'

In [ ]:
# Токенизатор предложений
nltk.download('punkt')
# Теггер частей речи
nltk.download('averaged_perceptron_tagger')

# Функция для лемматизации текста
def lemmatize_text(text):
    # Токенизация текста
    tokens = nltk.word_tokenize(text)

    # Тегирование части речи
    tagged = nltk.pos_tag(tokens)

    # Применение лемматизации ко всем словам в тексте
    lemm_text = ' '.join([lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in tagged])

    return lemm_text

# Функция для преобразования тегов частей речи в формат WordNet
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return 'a'  # прилагательное
    elif tag.startswith('V'):
        return 'v'  # глагол
    elif tag.startswith('N'):
        return 'n'  # существительное
    elif tag.startswith('R'):
        return 'r'  # наречие
    else:
        return 'n'  # по умолчанию - существительное

df['lemm_text'] = df['text'].apply(lemmatize_text)

df['lemm_text'][0]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


'explanation why the edits make under my username hardcore metallica fan be revert they werent vandalism just closure on some gas after i vote at new york doll fac and please dont remove the template from the talk page since im retire now'

## Обучение

Создадаем счётчик, указав в нём стоп-слова

In [ ]:
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [ ]:
df.head()

,text,toxic,lemm_text
0,explanation why the edits made under my userna...,0,explanation why the edits make under my userna...
1,daww he matches this background colour im seem...,0,daww he match this background colour im seemin...
2,hey man im really not trying to edit war its j...,0,hey man im really not try to edit war it just ...
3,more i cant make any real suggestions on impro...,0,more i cant make any real suggestion on improv...
4,you sir are my hero any chance you remember wh...,0,you sir be my hero any chance you remember wha...


Выделяем фичи и таргет

In [ ]:
df_ = df.copy()
X = df_.drop(['text' ,'toxic'], axis = 1)
y = df_['toxic']

Делим данные на тренировочную и тестовую выборку

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
X_train.shape

(127433, 1)

In [ ]:
y_train.shape

(127433,)

Производим векторизацию данных

In [ ]:
column_transformer = make_column_transformer((TfidfVectorizer(stop_words='english'), 'lemm_text'),
    remainder='passthrough'
)

pipe_dt = make_imblearn_pipeline(column_transformer, RandomUnderSampler(random_state=42), DecisionTreeClassifier(random_state=42))
pipe_knn = make_imblearn_pipeline(column_transformer, RandomUnderSampler(random_state=42), KNeighborsClassifier())
pipe_lr = make_imblearn_pipeline(column_transformer, RandomUnderSampler(random_state=42), LogisticRegression(random_state=42))
pipe_rf = make_imblearn_pipeline(column_transformer, RandomUnderSampler(random_state=42), RandomForestClassifier(random_state=42))
pipe_cb = make_imblearn_pipeline(column_transformer, EditedNearestNeighbours(n_neighbors=15), CatBoostClassifier(random_state=42))

param_grid_cb = {
    'catboostclassifier__learning_rate': [0.06],
    'catboostclassifier__depth': [5],
    'catboostclassifier__l2_leaf_reg': [10],
    'catboostclassifier__border_count': [128],
    'catboostclassifier__n_estimators': [900]
}

param_grid_dt = {
    'decisiontreeclassifier__max_depth': range(2, 31, 2),
    'decisiontreeclassifier__max_features': range(2, 13, 2)
}

param_grid_knn = {
    'kneighborsclassifier__n_neighbors': range(1, 200, 5),
    'kneighborsclassifier__metric': ['euclidean', 'cityblocks']
}

param_grid_lr = {
    'logisticregression__C': [0.1, 1, 3, 5, 7, 10],
    'logisticregression__penalty': ['l1', 'l2', 'elasticnet', 'None'],
    'logisticregression__solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
}

param_grid_rf = {
    'randomforestclassifier__min_samples_leaf': range(1, 20),
    'randomforestclassifier__max_depth': range(2, 30),
    'randomforestclassifier__min_samples_split': range(2, 20),
    'randomforestclassifier__max_features': range(2, 13),
    'randomforestclassifier__n_estimators': range(10, 101),
    'randomforestclassifier__criterion': ['gini', 'entropy', 'log_loss']
}

randomized_search_dt = RandomizedSearchCV(pipe_dt, param_grid_dt, cv=3, n_iter=10, scoring='f1', random_state=42, n_jobs=-1)
randomized_search_knn = RandomizedSearchCV(pipe_knn, param_grid_knn, cv=3, n_iter=10, scoring='f1', random_state=42, n_jobs=-1)
randomized_search_lr = RandomizedSearchCV(pipe_lr, param_grid_lr, cv=3, n_iter=10, scoring='f1', random_state=42, n_jobs=-1)
randomized_search_rf = RandomizedSearchCV(pipe_rf, param_grid_rf, cv=3, n_iter=10, scoring='f1', random_state=42, n_jobs=-1)
randomized_search_cb = RandomizedSearchCV(pipe_cb, param_grid_cb, cv=3, n_iter=1, scoring='f1', random_state=42, n_jobs=-1)

randomized_search_dt.fit(X_train, y_train)
randomized_search_knn.fit(X_train, y_train)
randomized_search_lr.fit(X_train, y_train)
randomized_search_rf.fit(X_train, y_train)
randomized_search_cb.fit(X_train, y_train, catboostclassifier__verbose=False)

print(f'Лучшая модель Decision Tree и её параметры:\n{randomized_search_dt.best_estimator_}\n\nМетрика лучшей модели Decision Tree на кросс-валидации:\n{randomized_search_dt.best_score_}\n\n')
print(f'Лучшая модель KNN и её параметры:\n{randomized_search_knn.best_estimator_}\n\nМетрика лучшей модели KNN на кросс-валидации:\n{randomized_search_knn.best_score_}\n\n')
print(f'Лучшая модель Logistic Regression и её параметры:\n{randomized_search_lr.best_estimator_}\n\nМетрика лучшей модели Logistic Regression на кросс-валидации:\n{randomized_search_lr.best_score_}\n\n')
print(f'Лучшая модель Random Forest и её параметры:\n{randomized_search_rf.best_estimator_}\n\nМетрика лучшей модели Random Forest на кросс-валидации:\n{randomized_search_rf.best_score_}\n\n')
print(f'Лучшая модель CatBoost и её параметры:\n{randomized_search_cb.best_estimator_.get_params()}\n\nМетрика лучшей модели CatBoost на кросс-валидации:\n{randomized_search_cb.best_score_}\n\n')

Лучшая модель Decision Tree и её параметры:
Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('tfidfvectorizer',
                                                  TfidfVectorizer(stop_words='english'),
                                                  'lemm_text')])),
                ('randomundersampler', RandomUnderSampler(random_state=42)),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(max_depth=22, max_features=12,
                                        random_state=42))])

Метрика лучшей модели Decision Tree на кросс-валидации:
0.18668591345653276


Лучшая модель KNN и её параметры:
Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('tfidfvectorizer',
                                                  TfidfVectorizer(stop_words='english'),
             

In [ ]:
y_pred = randomized_search_cb.best_estimator_.predict(X_test)
print('Результаты лучшей модели на тестовом наборе данных:')
print('F1-score:', f1_score(y_test, y_pred))

Результаты лучшей модели на тестовом наборе данных:
F1-score: 0.7574216827948171


## Выводы

<h3>Подготовка</h3>

Данные представленны в виде таблицы dataframe с 2 колонками:

-1-ая это текст представленный комментариями в типе данных object

-2-ая колонка это целевой признак бинарной классификации представленный в 1 или 0 как тип данных int

Есть дисбаланс классов на 9 обычных комментариев встречается 1 'токсичный'

Пропусков и явных дубликатов в данных нет

Проведенна лемматизация, очистка текста

<h3>Обучение</h3>

Выполнена векторизация данных

Из алгоритмов DecisionTreeClassifier, KNeighborsClassifier, LogisticRegression, CatBoostClassifier найденна лучшая модель и ее параметры

Лучшая модель и её параметры:

CatBoostClassifier(random_state=42, n_estimator=900, learning_rate=0.06, l2_leaf_reg=10, depth=5, border_count=128)

Метрика лучшей модели на кросс-валидации: 0.753384395897148

F1 на тесте: 0.7574216827948171